In [2]:
import matplotlib.animation as animation
import numpy as np
from numpy.linalg import eigh, eig
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly import graph_objects as go

In [4]:
class Map:
    def __init__(self, src, x, y):
        raw_data = np.loadtxt(src, delimiter='\t')
        self.shape = (x, y)
        self.raman_shift = raw_data.T[0][2:]
        self.data_matrix = raw_data[2:].T[1:]
        self.data = pd.DataFrame(self.data_matrix, columns = ['RamanShift'].extend(np.arange(0, x*y, 1)))
        self.map = np.array([raw_data[0][1:], raw_data[1][1:]]).T
        print(self.map.head())

        self.cov_matrix = np.cov(self.data_matrix)
        print(self.cov_matrix.shape)
        
        eigenvalues, eigenvectors = eigh(self.cov_matrix)
        idx = np.flip(eigenvalues.argsort())
        self.weights = eigenvalues[idx] / sum(eigenvalues)
        self.vectors = eigenvectors.T[idx]
        # for i in range(len(self.vectors)):
        #     self.vectors[i]/=np.dot(self.vectors[i], self.vectors[i])**.5
    
    def getBaseVectors(self, count):
        feature_matrix = self.vectors.copy()[:count]
        z_scale = []
        data_matrix = [x['spectrum'] for x in self.map]
        z_matrix = np.dot(feature_matrix, data_matrix)
        # for i in range(len(z_matrix)):
        #     _size = np.dot(z_matrix[i], z_matrix[i])**.5
        #     z_matrix[i] /= _size
        #     feature_matrix = feature_matrix.T
        #     feature_matrix[i] *= _size
        #     feature_matrix = feature_matrix.T
        #     z_scale.append(_size)
        return z_matrix
    
    def plotCovMatrix(self):
        fig = px.imshow(self.cov_matrix, title = 'Macierz kowariancji')
        return fig
    
    def plotWeights(self):
        fig = px.bar(self.weights, title = 'Wagi')
        return fig
    
    def plotSpectrum(self, id):
        fig = px.line(x = self.raman_shift, y = self.map[id]['spectrum'])
        return fig
        
    def locToId(self, x, y):
        return np.argmin(list(map(lambda e: abs(e['x'] - x) + abs(e['y'] - y), self.map)))
    
    
        
map_a = Map('../data/GA25_10B_532nm_100%_1x3sec_x100_xc1200_A_trojkat1_mapa_10x10um_step_0.3um_data.txt')

In [43]:
# map_a.plotCovMatrix().show()
# map_a.plotWeights().show()
# print(map_a.locToId(0.3,0))
# print(map_a.raman_shift)
# map_a.plotSpectrum(5).show()

1


In [6]:
z = map_a.getBaseVectors(10)
print(z.shape)
r = np.dot(map_a.vectors[:10].T, z)
px.line(x = map_a.raman_shift, y = z[0]).show()
px.line(x = map_a.raman_shift, y = r[0]).show()
map_a.plotSpectrum(0)


(10, 1011)
